<a href="https://colab.research.google.com/github/siriwatsc-debug/DADS5001-HW2/blob/main/DADS5001_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Link : Premier League Web
https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691

In [ ]:
Insight
# 1. ทีมที่นำตาราง
# 2. ทีมที่ทำประตูได้มากที่สุด
# 3. ทีมที่รับประตูน้อยที่สุด
# 4. ทีมที่มี xG diff สูงสุด (โชคดี/ทำได้เกินคาด)
# 5. ทีมที่มี xGA diff ติดลบมาก (โชคดีในการป้องกัน)
# 6. ทีมที่มี Goal Difference ดีที่สุด
# 7. ทีมที่มีผู้ชมเยอะที่สุด
# 8. ทีมที่มีอัตราชนะสูงสุด
# 9. ทีมที่ดิ้นรนหนัก (xGD ดีแต่คะแนนน้อย)

Visualization
# 1. ตารางคะแนน Top 10
# 2. Goals Scored vs Conceded (Bubble Chart)
# 3. Actual Goals vs Expected Goals (xG)
# 4. Win/Draw/Loss Distribution
# 5. xG Difference (Over/Under Performance)
# 6. Attendance vs Performance
# 7. Goal Difference Ranking  (ผลต่างประตูได้เสีย by team)
# 8. Actual Goal Difference


In [35]:
# Homework 2
# DADS5001 : Tools and Programing (2nd Half)
# Siriwat Carawukh 6720422017

import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import re

# Scrape data from Web
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    try:
        # Read table from web url
        tables = pd.read_html(url)
        # Check number of tables
        print(f"Found total table {len(tables)}")
        # Define table[0] for Premier League Table Statistic
        df = tables[0]
        # Show dataframe
        print(df)

        if df :
            print(f"✅ ดึงข้อมูลสำเร็จ! พบ {len(df)} ทีม")
            print(f"📊 Columns: {df.columns.tolist()}")
            return df

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 ใช้ข้อมูลจากภาพที่แนบมา")

    return create_data_from_image()

def create_data_from_image():
    """
    สร้างข้อมูลจากภาพที่แนบมา (ข้อมูลจริง Premier League 2024-25)
    """
    data = {
        'Rk': list(range(1, 21)),
        'Squad': ['Arsenal', 'Bournemouth', 'Tottenham', 'Sunderland', 'Manchester City',
                 'Manchester Utd', 'Liverpool', 'Aston Villa', 'Chelsea', 'Crystal Palace',
                 'Brentford', 'Newcastle Utd', 'Brighton', 'Everton', 'Leeds United',
                 'Burnley', 'Fulham', 'Nottingham Forest', 'West Ham', 'Wolves'],
        'MP': [9] * 20,
        'W': [7, 5, 5, 5, 5, 5, 5, 4, 4, 3, 4, 3, 3, 3, 3, 3, 2, 1, 1, 0],
        'D': [1, 3, 2, 2, 1, 1, 0, 3, 2, 4, 1, 3, 3, 2, 2, 1, 2, 2, 1, 2],
        'L': [1, 1, 2, 2, 3, 3, 4, 2, 3, 2, 4, 3, 3, 4, 4, 5, 5, 6, 7, 7],
        'GF': [16, 16, 17, 11, 17, 15, 16, 9, 17, 12, 14, 9, 14, 9, 9, 12, 9, 5, 7, 7],
        'GA': [3, 11, 7, 7, 7, 14, 14, 8, 11, 9, 14, 8, 15, 12, 14, 17, 14, 17, 20, 19],
        'GD': [13, 5, 10, 4, 10, 1, 2, 1, 6, 3, 0, 1, -1, -3, -5, -5, -5, -12, -13, -12],
        'Pts': [22, 18, 17, 17, 16, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 8, 5, 4, 2],
        'Pts/MP': [2.44, 2.00, 1.89, 1.89, 1.78, 1.78, 1.67, 1.67, 1.56, 1.44,
                  1.44, 1.33, 1.33, 1.22, 1.22, 1.11, 0.89, 0.56, 0.44, 0.22],
        'xGD_90': [1.04, 1.11, 1.26, 0.90, 0.66, 1.62, 0.42, 0.03, 0.59, 1.96,
                  1.49, 0.78, 1.48, 1.37, 1.00, 0.76, -0.42, -0.80, -0.89, -0.25],
        'xG': [14.6, 11.0, 10.3, 8.6, 15.6, 16.2, 15.8, 7.1, 13.4, 17.6, 13.4, 11.9, 13.2, 12.3, 11.5, 6.8, 9.0, 9.1, 7.6, 9.2],
        'xGA': [5.2, 11.1, 10.5, 10.4, 9.0, 13.6, 12.0, 11.8, 13.1, 11.7, 12.2, 7.0, 11.8, 14.0, 9.0, 18.1, 12.8, 14.7, 14.8, 11.4],
        'Attendance': [60144, 11164, 61039, 46335, 52311, 73965, 60362, 41242,
                      39640, 25068, 17014, 52183, 31518, 51883, 36722, 21521,
                      27172, 30221, 62457, 30231],
        'Top_Scorer': ['Viktor Gyökeres - 3', 'Antoine Semenyo - 6', 'Richarlison, Nicky van de Ven - 3',
                      'Wilson Isidor - 4', 'Erling Haaland - 11', 'Bryan Mbeumo - 4',
                      'Mohamed Salah, Cody Gakpo - 3', 'Emi Buendía, Donyell Malen - 2',
                      'Enzo Fernández, Moisés Caicedo - 3', 'Jean-Philippe Mateta - 5',
                      'Thiago - 6', 'Nick Woltemade - 4', 'Danny Welbeck - 5',
                      'Iliman Ndiaye - 3', 'Joe Rodon, Noah Okafor - 2', 'Jaidon Anthony - 4',
                      'Raúl Jiménez, Alex Iwobi - 1', 'Chris Wood - 2', 'Jarrod Bowen - 3',
                      'Hwang Hee-chan, Santiago Bueno - 1'],
    }

    df = pd.DataFrame(data)

    # คำนวณ xGD รวมจาก xGD/90
    df['xGD'] = (df['xGD_90'] * df['MP']).round(1)

    # คำนวณค่าเพิ่มเติม
    df['Win_Rate'] = (df['W'] / df['MP'] * 100).round(1)
    df['xG_diff'] = (df['GF'] - df['xG']).round(1)
    df['xGA_diff'] = (df['GA'] - df['xGA']).round(1)
    df['Goals_per_Match'] = (df['GF'] / df['MP']).round(2)
    df['Conceded_per_Match'] = (df['GA'] / df['MP']).round(2)
    # คำนวณ Performance Gap (GD vs xGD)
    df['Performance Gap'] = (df['GD'] - df['xGD']).round(1)

    return df

def analyze_insights(df):
    """
    วิเคราะห์หา insights ที่น่าสนใจจากข้อมูล
    """
    insights = []

    # 1. ทีมที่นำตาราง
    top_team = df.iloc[0]
    insights.append(f"🏆 ทีมอันดับ 1: {top_team['Squad']} ({top_team['Pts']} คะแนน, {top_team['W']} ชนะ)")

    # 2. ทีมที่ทำประตูได้มากที่สุด
    top_scorer_team = df.loc[df['GF'].idxmax()]
    insights.append(f"⚽ ทำประตูมากสุด: {top_scorer_team['Squad']} ({top_scorer_team['GF']} ประตู)")

    # 3. ทีมที่รับประตูน้อยที่สุด
    best_defense = df.loc[df['GA'].idxmin()]
    insights.append(f"🛡️ ป้องกันดีสุด: {best_defense['Squad']} (เสีย {best_defense['GA']} ประตูเท่านั้น)")

    # 4. ทีมที่มี Performance Gap สูงสุด (ทำได้ดีกว่าที่คาด)
    overperforming_team = df.loc[df['Performance Gap'].idxmax()]
    if overperforming_team['Performance Gap'] > 4:
        insights.append(f"⚡ Overperforming: {overperforming_team['Squad']} (GD +{overperforming_team['GD']} vs xGD +{overperforming_team['xGD']:.1f})")

    # 5. ทีมที่มี Performance Gap ต่ำสุด (ทำได้แย่กว่าที่คาด)
    underperforming_team = df.loc[df['Performance Gap'].idxmin()]
    if underperforming_team['Performance Gap'] < -4:
        insights.append(f"📉 Underperforming: {underperforming_team['Squad']} (GD {underperforming_team['GD']} vs xGD {underperforming_team['xGD']:.1f})")

    # 6. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.loc[df['GD'].idxmax()]
    insights.append(f"📊 ผลต่างประตูดีสุด: {best_gd['Squad']} (+{best_gd['GD']})")

    # 7. ทีมที่มีผู้ชมเยอะที่สุด
    most_attendance = df.loc[df['Attendance'].idxmax()]
    insights.append(f"👥 ผู้ชมเฉลี่ยสูงสุด: {most_attendance['Squad']} ({most_attendance['Attendance']:,} คน)")

    # 8. ทีมที่มีอัตราชนะสูงสุด
    best_win_rate = df.loc[df['Win_Rate'].idxmax()]
    insights.append(f"📈 อัตราชนะสูงสุด: {best_win_rate['Squad']} ({best_win_rate['Win_Rate']:.1f}%)")

    # 9. ทีมที่ดิ้นรนหนัก (คะแนนน้อย)
    struggling_teams = df.nsmallest(3, 'Pts')
    insights.append(f"😥 ทีมท้ายตาราง: {', '.join(struggling_teams['Squad'].tolist())} (คะแนนรวม {struggling_teams['Pts'].sum()} คะแนน)")

    return insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ใน Subplot 3x3 พร้อม Key Insights
    """

    # สร้าง subplot 3 แถว 3 คอลัมน์ (ช่องที่ 9 เป็น table สำหรับ Key Insights)
    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=(
            '🏆 Top 10 Teams',
            '⚽ Goals Scored vs Conceded',
            '📈 Actual vs Expected Goals',
            '📊 Win/Draw/Loss Distribution',
            '🎯 Performance Gap (GD - xGD)',
            '👥 Attendance vs Performance',
            '🔢 Goal Difference Ranking',
            '⚽ GD vs xGD Performance',
            '🔍 KEY INSIGHTS'
        ),
        specs=[
            [{"type": "bar"}, {"type": "scatter"}, {"type": "bar"}],
            [{"type": "bar"}, {"type": "bar"}, {"type": "scatter"}],
            [{"type": "bar"}, {"type": "scatter"}, {"type": "table"}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )

    # 1. ตารางคะแนน Top 10 (แนวนอน)
    top10 = df.head(10).sort_values('Pts', ascending=True)
    fig.add_trace(
        go.Bar(
            y=top10['Squad'],
            x=top10['Pts'],
            orientation='h',
            marker=dict(color=top10['Pts'], colorscale='Viridis'),
            text=top10['Pts'],
            textposition='outside',
            name='Points',
            showlegend=False,
            hovertemplate='<b>%{y}</b><br>คะแนน: %{x}<br><extra></extra>'
        ),
        row=1, col=1
    )

    # 2. Goals Scored vs Conceded (Bubble)
    fig.add_trace(
        go.Scatter(
            x=df['GF'],
            y=df['GA'],
            mode='markers+text',
            marker=dict(
                size=df['Pts']*2,
                color=df['Pts'],
                colorscale='RdYlGn',
                showscale=False,
                line=dict(width=1, color='white')
            ),
            text=df['Squad'].str[:3],
            textposition='middle center',
            textfont=dict(size=7),
            name='Teams',
            showlegend=False,
            hovertext=df['Squad'],
            hovertemplate='<b>%{hovertext}</b><br>GF: %{x}<br>GA: %{y}<extra></extra>'
        ),
        row=1, col=2
    )
    # เพิ่มเส้นแนวทแยง
    fig.add_trace(
        go.Scatter(
            x=[0, 25], y=[0, 25],
            mode='lines',
            line=dict(color='gray', dash='dash', width=1),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=1, col=2
    )

    # 3. Actual vs Expected Goals
    df_sorted = df.sort_values('Pts', ascending=False).head(10)
    fig.add_trace(
        go.Bar(
            x=df_sorted['Squad'],
            y=df_sorted['GF'],
            name='Actual Goals',
            marker_color='steelblue',
            showlegend=True
        ),
        row=1, col=3
    )
    fig.add_trace(
        go.Scatter(
            x=df_sorted['Squad'],
            y=df_sorted['xG'],
            mode='markers',
            name='xG',
            marker=dict(size=8, color='red', symbol='diamond'),
            showlegend=True
        ),
        row=1, col=3
    )

    # 4. Win/Draw/Loss Distribution
    df_sorted = df.sort_values('Pts', ascending=False).head(10)
    fig.add_trace(
        go.Bar(x=df_sorted['Squad'], y=df_sorted['W'], name='Win', marker_color='green'),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=df_sorted['Squad'], y=df_sorted['D'], name='Draw', marker_color='orange'),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=df_sorted['Squad'], y=df_sorted['L'], name='Loss', marker_color='red'),
        row=2, col=1
    )

    # 5. Performance Gap
    df_sorted = df.sort_values('Performance Gap', ascending=False).head(10)
    colors = ['green' if x > 0 else 'red' for x in df_sorted['Performance Gap']]
    fig.add_trace(
        go.Bar(
            x=df_sorted['Squad'],
            y=df_sorted['Performance Gap'],
            marker_color=colors,
            text=df_sorted['Performance Gap'].round(1),
            textposition='outside',
            showlegend=False,
            hovertemplate='<b>%{x}</b><br>GD - xGD: %{y:.1f}<extra></extra>'
        ),
        row=2, col=2
    )

    # 6. Attendance vs Performance
    fig.add_trace(
        go.Scatter(
            x=df['Attendance'],
            y=df['Pts'],
            mode='markers+text',
            marker=dict(
                size=df['GF']*2,
                color=df['GD'],
                colorscale='RdYlGn',
                showscale=False,
                line=dict(width=1, color='white')
            ),
            text=df['Squad'].str[:3],
            textposition='top center',
            textfont=dict(size=7),
            showlegend=False,
            hovertext=df['Squad'],
            hovertemplate='<b>%{hovertext}</b><br>Attendance: %{x:,}<br>Points: %{y}<extra></extra>'
        ),
        row=2, col=3
    )

    # 7. Goal Difference Ranking
    df_sorted = df.sort_values('GD', ascending=False).head(10)
    fig.add_trace(
        go.Bar(
            x=df_sorted['Squad'],
            y=df_sorted['GD'],
            marker=dict(color=df_sorted['GD'], colorscale='RdYlGn'),
            text=df_sorted['GD'],
            textposition='outside',
            showlegend=False
        ),
        row=3, col=1
    )

    # 8. GD vs xGD Scatter
    fig.add_trace(
        go.Scatter(
            x=df['xGD'],
            y=df['GD'],
            mode='markers+text',
            marker=dict(
                size=df['Pts']*2,
                color=df['Performance Gap'],
                colorscale='RdYlGn',
                showscale=False,
                line=dict(width=1, color='white')
            ),
            #title='⚽ ประสิทธิภาพพรีเมียร์ลีก: ผลต่างประตูจริง (GD) vs. ผลต่างประตูคาดหวัง (xGD)',
            text=df['Squad'].str[:3],
            textposition='top center',
            textfont=dict(size=7),
            showlegend=False,
            hovertext=df['Squad'],
            hovertemplate='<b>%{hovertext}</b><br>xGD: %{x:.1f}<br>Actual GD: %{y}<extra></extra>'
        ),
        row=3, col=2
    )
    # เพิ่มเส้นแนวทแยง
    min_val = min(df['xGD'].min(), df['GD'].min()) - 2
    max_val = max(df['xGD'].max(), df['GD'].max()) + 2
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            mode='lines',
            line=dict(color='gray', dash='dash', width=1),
            showlegend=False,
            hoverinfo='skip'
        ),
        row=3, col=2
    )

    # 9. Key Insights Table
    insights = analyze_insights(df)

    # สร้างตาราง Key Insights
    insight_labels = []
    insight_values = []

    for i, insight in enumerate(insights, 1):
        # แยก label และ value
        parts = insight.split(': ', 1)
        if len(parts) == 2:
            insight_labels.append(parts[0])
            insight_values.append(parts[1])
        else:
            insight_labels.append(f"Insight {i}")
            insight_values.append(insight)

    fig.add_trace(
        go.Table(
            header=dict(
                values=['<b>Category</b>', '<b>Detail</b>'],
                fill_color='#2C3E50',
                align='left',
                font=dict(size=11, color='white', family='Arial Black'),
                height=30
            ),
            cells=dict(
                values=[insight_labels, insight_values],
                fill_color=[['#ECF0F1', '#D5DBDB'] * len(insight_labels)],
                align='left',
                font=dict(size=10, family='Arial'),
                height=25
            ),
            columnwidth=[0.4, 0.6]
        ),
        row=3, col=3
    )

    # Update layout
    fig.update_layout(
        height=1400,
        showlegend=True,
        title_text="⚽ Premier League 2024-25 Complete Analysis Dashboard",
        title_font_size=20,
        barmode='stack',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Update axes
    fig.update_xaxes(title_text="Points", row=1, col=1)
    fig.update_yaxes(title_text="Team", row=1, col=1)

    fig.update_xaxes(title_text="Goals For", row=1, col=2)
    fig.update_yaxes(title_text="Goals Against", row=1, col=2)

    fig.update_xaxes(title_text="Team", tickangle=-45, row=1, col=3)
    fig.update_yaxes(title_text="Goals", row=1, col=3)

    fig.update_xaxes(title_text="Team", tickangle=-45, row=2, col=1)
    fig.update_yaxes(title_text="Matches", row=2, col=1)

    fig.update_xaxes(title_text="Team", tickangle=-45, row=2, col=2)
    fig.update_yaxes(title_text="GD - xGD", row=2, col=2)

    fig.update_xaxes(title_text="Attendance", row=2, col=3)
    fig.update_yaxes(title_text="Points", row=2, col=3)

    fig.update_xaxes(title_text="Team", tickangle=-45, row=3, col=1)
    fig.update_yaxes(title_text="Goal Difference", row=3, col=1)

    fig.update_xaxes(title_text="xGD", row=3, col=2)
    fig.update_yaxes(title_text="Actual GD", row=3, col=2)

    return fig

# Main execution (แก้ไขส่วนนี้เพื่อแสดง subplot เดียว)
if __name__ == "__main__":
    print("="*70)
    print("⚽ PREMIER LEAGUE DATA ANALYSIS")
    print("="*70)

    # ใช้ URL ที่พยายามจะ Scraping แต่จะ Fallback ไปใช้ข้อมูลจากภาพ
    url = "https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691"

    print("\n🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n" + "="*70)
    print("🔍 KEY INSIGHTS - ข้อมูลเชิงลึกที่น่าสนใจ")
    print("="*70)

    insights = analyze_insights(df)
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")

    print("\n" + "="*70)
    print("📊 สถิติเพิ่มเติม")
    print("="*70)
    print(f"• ค่าเฉลี่ยประตูต่อเกมทั้งลีก: {df['Goals_per_Match'].mean():.2f}")
    print(f"• ทีมที่ทำประตูมากที่สุด 3 อันดับ: {', '.join(df.nlargest(3, 'GF')['Squad'].tolist())}")
    print(f"• ทีมที่เสียประตูน้อยที่สุด 3 อันดับ: {', '.join(df.nsmallest(3, 'GA')['Squad'].tolist())}")
    print(f"• ทีมที่มีผู้ชมมากที่สุด: {df.loc[df['Attendance'].idxmax(), 'Squad']} ({df['Attendance'].max():,} คน)")

    print("\n" + "="*70)
    print("📈 กำลังสร้าง Dashboard Subplot...")
    print("="*70)

    # สร้างและแสดง subplot รวม 9 ช่อง
    fig = create_visualizations(df)
    fig.show()

    # บันทึกข้อมูล
    df.to_csv('premier_league_stats_2024_25.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ 'premier_league_stats_2024_25.csv' เรียบร้อยแล้ว!")

    # สรุปทีมที่น่าสนใจ
    print("\n" + "="*70)
    print("🌟 ทีมที่น่าจับตามอง")
    print("="*70)

    # ทีมที่ทำได้ดีกว่าที่คาด (Overperformers)
    overperformers = df[df['Performance Gap'] > 4].sort_values('Performance Gap', ascending=False)
    if not overperformers.empty:
        print("\n🚀 ทีมที่ทำได้เหนือความคาด (GD > xGD):")
        for _, team in overperformers.iterrows():
            print(f"   • {team['Squad']}: ส่วนต่าง +{team['Performance Gap']:.1f} (GD={team['GD']} vs xGD={team['xGD']:.1f})")

    # ทีมที่น่าเป็นห่วง (Underperformers)
    underperformers = df[df['Performance Gap'] < -4].sort_values('Performance Gap')
    if not underperformers.empty:
        print("\n⚠️ ทีมที่ยังทำได้ไม่ดีเท่าที่ควร (GD < xGD):")
        for _, team in underperformers.iterrows():
            print(f"   • {team['Squad']}: ส่วนต่าง {team['Performance Gap']:.1f} (GD={team['GD']} vs xGD={team['xGD']:.1f})")

    print("\n" + "="*70)
    print("📋 ตารางข้อมูลทั้งหมด")
    print("="*70)
    print(df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA', 'xGD', 'Performance Gap']].to_string(index=False))

⚽ PREMIER LEAGUE DATA ANALYSIS

🔄 กำลังดึงข้อมูล Premier League...
Found total table 24
    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13


✅ บันทึกข้อมูลเป็นไฟล์ 'premier_league_stats_2024_25.csv' เรียบร้อยแล้ว!

🌟 ทีมที่น่าจับตามอง

🚀 ทีมที่ทำได้เหนือความคาด (GD > xGD):
   • Manchester City: ส่วนต่าง +4.1 (GD=10 vs xGD=5.9)

⚠️ ทีมที่ยังทำได้ไม่ดีเท่าที่ควร (GD < xGD):
   • Everton: ส่วนต่าง -15.3 (GD=-3 vs xGD=12.3)
   • Crystal Palace: ส่วนต่าง -14.6 (GD=3 vs xGD=17.6)
   • Brighton: ส่วนต่าง -14.3 (GD=-1 vs xGD=13.3)
   • Leeds United: ส่วนต่าง -14.0 (GD=-5 vs xGD=9.0)
   • Manchester Utd: ส่วนต่าง -13.6 (GD=1 vs xGD=14.6)
   • Brentford: ส่วนต่าง -13.4 (GD=0 vs xGD=13.4)
   • Burnley: ส่วนต่าง -11.8 (GD=-5 vs xGD=6.8)
   • Wolves: ส่วนต่าง -9.8 (GD=-12 vs xGD=-2.2)
   • Newcastle Utd: ส่วนต่าง -6.0 (GD=1 vs xGD=7.0)
   • Bournemouth: ส่วนต่าง -5.0 (GD=5 vs xGD=10.0)
   • West Ham: ส่วนต่าง -5.0 (GD=-13 vs xGD=-8.0)
   • Nottingham Forest: ส่วนต่าง -4.8 (GD=-12 vs xGD=-7.2)
   • Sunderland: ส่วนต่าง -4.1 (GD=4 vs xGD=8.1)

📋 ตารางข้อมูลทั้งหมด
 Rk             Squad  MP  W  D  L  GF  GA  GD  Pts   xG  xGA  xGD  Perfor

In [29]:
# Work as individual plots


import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import re

# Scrape data from Web
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    try:
        # Read table from web url
        tables = pd.read_html(url)
        # Check number of tables
        print(f"Found total table {len(tables)}")
        # Define table[0] for Premier League Table Statistic
        df = tables[0]
        # Show dataframe
        print(df)

        if df :
            print(f"✅ ดึงข้อมูลสำเร็จ! พบ {len(df)} ทีม")
            print(f"📊 Columns: {df.columns.tolist()}")
            return df

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 ใช้ข้อมูลจากภาพที่แนบมา")

    return create_data_from_image()

def create_data_from_image():
    """
    สร้างข้อมูลจากภาพที่แนบมา (ข้อมูลจริง Premier League 2024-25)
    """
    data = {
        'Rk': list(range(1, 21)),
        'Squad': ['Arsenal', 'Bournemouth', 'Tottenham', 'Sunderland', 'Manchester City',
                 'Manchester Utd', 'Liverpool', 'Aston Villa', 'Chelsea', 'Crystal Palace',
                 'Brentford', 'Newcastle Utd', 'Brighton', 'Everton', 'Leeds United',
                 'Burnley', 'Fulham', 'Nottingham Forest', 'West Ham', 'Wolves'],
        'MP': [9] * 20,
        'W': [7, 5, 5, 5, 5, 5, 5, 4, 4, 3, 4, 3, 3, 3, 3, 3, 2, 1, 1, 0],
        'D': [1, 3, 2, 2, 1, 1, 0, 3, 2, 4, 1, 3, 3, 2, 2, 1, 2, 2, 1, 2],
        'L': [1, 1, 2, 2, 3, 3, 4, 2, 3, 2, 4, 3, 3, 4, 4, 5, 5, 6, 7, 7],
        'GF': [16, 16, 17, 11, 17, 15, 16, 9, 17, 12, 14, 9, 14, 9, 9, 12, 9, 5, 7, 7],
        'GA': [3, 11, 7, 7, 7, 14, 14, 8, 11, 9, 14, 8, 15, 12, 14, 17, 14, 17, 20, 19],
        'GD': [13, 5, 10, 4, 10, 1, 2, 1, 6, 3, 0, 1, -1, -3, -5, -5, -5, -12, -13, -12],
        'Pts': [22, 18, 17, 17, 16, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 8, 5, 4, 2],
        'Pts/MP': [2.44, 2.00, 1.89, 1.89, 1.78, 1.78, 1.67, 1.67, 1.56, 1.44,
                  1.44, 1.33, 1.33, 1.22, 1.22, 1.11, 0.89, 0.56, 0.44, 0.22],
        # ใช้ค่า xGD/90 จากภาพ แล้วคูณ MP=9 เพื่อให้ได้ xGD รวม
        'xGD_90': [1.04, 1.11, 1.26, 0.90, 0.66, 1.62, 0.42, 0.03, 0.59, 1.96,
                  1.49, 0.78, 1.48, 1.37, 1.00, 0.76, -0.42, -0.80, -0.89, -0.25],
        # xG และ xGA จากภาพ (ค่าที่แปลงมาจาก Pts/MP และ xG/xGA/xGD ในภาพ)
        'xG': [14.6, 11.0, 10.3, 8.6, 15.6, 16.2, 15.8, 7.1, 13.4, 17.6, 13.4, 11.9, 13.2, 12.3, 11.5, 6.8, 9.0, 9.1, 7.6, 9.2],
        'xGA': [5.2, 11.1, 10.5, 10.4, 9.0, 13.6, 12.0, 11.8, 13.1, 11.7, 12.2, 7.0, 11.8, 14.0, 9.0, 18.1, 12.8, 14.7, 14.8, 11.4],
        'Attendance': [60144, 11164, 61039, 46335, 52311, 73965, 60362, 41242,
                      39640, 25068, 17014, 52183, 31518, 51883, 36722, 21521,
                      27172, 30221, 62457, 30231],
        'Top_Scorer': ['Viktor Gyökeres - 3', 'Antoine Semenyo - 6', 'Richarlison, Nicky van de Ven - 3',
                      'Wilson Isidor - 4', 'Erling Haaland - 11', 'Bryan Mbeumo - 4',
                      'Mohamed Salah, Cody Gakpo - 3', 'Emi Buendía, Donyell Malen - 2',
                      'Enzo Fernández, Moisés Caicedo - 3', 'Jean-Philippe Mateta - 5',
                      'Thiago - 6', 'Nick Woltemade - 4', 'Danny Welbeck - 5',
                      'Iliman Ndiaye - 3', 'Joe Rodon, Noah Okafor - 2', 'Jaidon Anthony - 4',
                      'Raúl Jiménez, Alex Iwobi - 1', 'Chris Wood - 2', 'Jarrod Bowen - 3',
                      'Hwang Hee-chan, Santiago Bueno - 1'],
    }

    df = pd.DataFrame(data)

    # คำนวณ xGD รวมจาก xGD/90
    df['xGD'] = (df['xGD_90'] * df['MP']).round(1)

    # คำนวณค่าเพิ่มเติม
    df['Win_Rate'] = (df['W'] / df['MP'] * 100).round(1)
    df['xG_diff'] = (df['GF'] - df['xG']).round(1)
    df['xGA_diff'] = (df['GA'] - df['xGA']).round(1)
    df['Goals_per_Match'] = (df['GF'] / df['MP']).round(2)
    df['Conceded_per_Match'] = (df['GA'] / df['MP']).round(2)
    # คำนวณ Performance Gap (GD vs xGD)
    df['Performance Gap'] = (df['GD'] - df['xGD']).round(1)

    return df

def analyze_insights(df):
    """
    วิเคราะห์หา insights ที่น่าสนใจจากข้อมูล
    """
    insights = []

    # 1. ทีมที่นำตาราง
    top_team = df.iloc[0]
    insights.append(f"🏆 ทีมอันดับ 1: {top_team['Squad']} ({top_team['Pts']} คะแนน, {top_team['W']} ชนะ)")

    # 2. ทีมที่ทำประตูได้มากที่สุด
    top_scorer_team = df.loc[df['GF'].idxmax()]
    insights.append(f"⚽ ทำประตูมากสุด: {top_scorer_team['Squad']} ({top_scorer_team['GF']} ประตู)")

    # 3. ทีมที่รับประตูน้อยที่สุด
    best_defense = df.loc[df['GA'].idxmin()]
    insights.append(f"🛡️ ป้องกันดีสุด: {best_defense['Squad']} (เสีย {best_defense['GA']} ประตูเท่านั้น)")

    # 4. ทีมที่มี Performance Gap สูงสุด (ทำได้ดีกว่าที่คาด)
    overperforming_team = df.loc[df['Performance Gap'].idxmax()]
    if overperforming_team['Performance Gap'] > 4:
        insights.append(f"⚡ Overperforming: **{overperforming_team['Squad']}** (GD +{overperforming_team['GD']} vs xGD +{overperforming_team['xGD']:.1f})")

    # 5. ทีมที่มี Performance Gap ต่ำสุด (ทำได้แย่กว่าที่คาด)
    underperforming_team = df.loc[df['Performance Gap'].idxmin()]
    if underperforming_team['Performance Gap'] < -4:
        insights.append(f"📉 Underperforming: **{underperforming_team['Squad']}** (GD {underperforming_team['GD']} vs xGD {underperforming_team['xGD']:.1f})")

    # 6. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.loc[df['GD'].idxmax()]
    insights.append(f"📊 ผลต่างประตูดีสุด: {best_gd['Squad']} (+{best_gd['GD']})")

    # 7. ทีมที่มีผู้ชมเยอะที่สุด
    most_attendance = df.loc[df['Attendance'].idxmax()]
    insights.append(f"👥 ผู้ชมเฉลี่ยสูงสุด: {most_attendance['Squad']} ({most_attendance['Attendance']:,} คน)")

    # 8. ทีมที่มีอัตราชนะสูงสุด
    best_win_rate = df.loc[df['Win_Rate'].idxmax()]
    insights.append(f"📈 อัตราชนะสูงสุด: {best_win_rate['Squad']} ({best_win_rate['Win_Rate']:.1f}%)")

    # 9. ทีมที่ดิ้นรนหนัก (คะแนนน้อย)
    struggling_teams = df.nsmallest(3, 'Pts')
    insights.append(f"😥 ทีมท้ายตาราง: {', '.join(struggling_teams['Squad'].tolist())} (คะแนนรวม {struggling_teams['Pts'].sum()} คะแนน)")

    return insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ที่หลากหลาย
    """

    # Initialize figure with subplots
    fig = make_subplots(
    rows=3, cols=3,
    column_widths=[0.3, 0.3, 0.4],
    row_heights=[0.3, 0.3, 0.4],
    specs=[[{"type": "bar"},        {"type": "bar"},    {"type": "bar"}],
           [{"type": "scatter"},    {"type": "bar"},    {"type": "scatter"}],
           [{"type": "bar"},        {"type": "scatter"},{"type": "bar"}]])


    # 1. ตารางคะแนน Top 10
    fig1 = go.Figure()

    top10 = df.head(10).sort_values('Pts', ascending=True)

    fig1.add_trace(go.Bar(
        y=top10['Squad'],
        x=top10['Pts'],
        orientation='h',
        marker=dict(
            color=top10['Pts'],
            colorscale='Viridis',
            showscale=True
        ),
        text=top10['Pts'],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>คะแนน: %{x}<br><extra></extra>'

    ))

    fig1.update_layout(
        title='🏆 Premier League - Top 10 Teams',
        xaxis_title='คะแนน (Points)',
        yaxis_title='ทีม',
        height=600,
        template='plotly_white'
    )

    # 2. Goals Scored vs Conceded (Bubble Chart)
    fig2 = px.scatter(df,
                      x='GF',
                      y='GA',
                      size='Pts',
                      color='Pts',
                      hover_name='Squad',
                      size_max=50,
                      color_continuous_scale='RdYlGn',
                      title='⚽ Goals Scored vs Goals Conceded (ขนาดวง = คะแนน)'
                      )

    fig2.add_shape(type='line', x0=0, y0=0, x1=25, y1=25,
                   line=dict(color='gray', dash='dash'),
                   name='เสมอ')

    fig2.update_layout(
        xaxis_title='ประตูที่ทำได้ (GF)',
        yaxis_title='ประตูที่เสีย (GA)',
        height=600
    )

    # 3. Actual Goals vs Expected Goals (xG)
    fig3 = go.Figure()

    # เรียงตามคะแนน
    df_sorted = df.sort_values('Pts', ascending=True)

    fig3.add_trace(go.Bar(
        name='Actual Goals (GF)',
        x=df_sorted['Squad'],
        y=df_sorted['GF'],
        marker_color='steelblue'
    ))

    fig3.add_trace(go.Scatter(
        name='Expected Goals (xG)',
        x=df_sorted['Squad'],
        y=df_sorted['xG'],
        mode='markers',
        marker=dict(size=10, color='red', symbol='diamond')
    ))

    fig3.update_layout(
        title='📈 Actual Goals vs Expected Goals (xG)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนประตู',
        xaxis_tickangle=-45,
        height=600,
        template='plotly_white',
        barmode='group'
    )

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    df_sorted = df.sort_values('Pts', ascending=False)

    fig4.add_trace(go.Bar(
        name='ชนะ (W)',
        x=df_sorted['Squad'],
        y=df_sorted['W'],
        marker_color='green'
    ))

    fig4.add_trace(go.Bar(
        name='เสมอ (D)',
        x=df_sorted['Squad'],
        y=df_sorted['D'],
        marker_color='orange'
    ))

    fig4.add_trace(go.Bar(
        name='แพ้ (L)',
        x=df_sorted['Squad'],
        y=df_sorted['L'],
        marker_color='red'
    ))

    fig4.update_layout(
        title='📊 การกระจายผลการแข่งขัน (W/D/L)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนเกม',
        barmode='stack',
        xaxis_tickangle=-45,
        height=600
    )

    # 5. xG Difference (Over/Under Performance)
    fig5 = go.Figure()

    df_sorted = df.sort_values('Performance Gap', ascending=True)

    colors = ['red' if x < 0 else 'green' for x in df_sorted['Performance Gap']]

    fig5.add_trace(go.Bar(
        x=df_sorted['Squad'],
        y=df_sorted['Performance Gap'],
        marker_color=colors,
        text=df_sorted['Performance Gap'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>GD - xGD: %{y:.1f}<extra></extra>'
    ))

    fig5.add_hline(y=0, line_dash="dash", line_color="black")

    fig5.update_layout(
        title='🎯 Performance Gap (GD - xGD) - ความแตกต่างจากที่คาด',
        xaxis_title='ทีม',
        yaxis_title='ผลต่าง (GD - xGD)',
        xaxis_tickangle=-45,
        height=600
    )

    # 6. Attendance vs Performance
    fig6 = px.scatter(df,
                     x='Attendance',
                     y='Pts',
                     size='GF',
                     color='GD',
                     hover_name='Squad',
                     title='👥 ผู้ชม vs ผลงาน (ขนาด = จำนวนประตูที่ทำได้)',
                     color_continuous_scale='RdYlGn')

    fig6.update_layout(
        xaxis_title='จำนวนผู้ชมเฉลี่ย',
        yaxis_title='คะแนน',
        height=600
    )

    # 7. Goal Difference Ranking
    fig7 = px.bar(df.sort_values('GD', ascending=True),
                  x='GD',
                  y='Squad',
                  orientation='h',
                  color='GD',
                  color_continuous_scale='RdYlGn',
                  title='🔢 Goal Difference (ผลต่างประตู)')

    fig7.update_layout(height=700)

    # ----------------------------------------------------------------------
    # 8. GD vs. xGD Scatter Plot (New Insight Plot)
    # ----------------------------------------------------------------------

    df['Performance Gap Label'] = df['Performance Gap'].apply(lambda x: 'Overperforming' if x > 0 else 'Underperforming')

    fig8 = px.scatter(
        df,
        x='xGD',
        y='GD',
        text='Squad',
        size='Pts', # ขนาดวงกลมตามคะแนน (Pts)
        color='Performance Gap', # สีตาม Performance Gap
        color_continuous_scale=px.colors.diverging.RdYlGn, # ใช้โทนสีเขียว-เหลือง-แดง
        title='⚽ Actual Goal Difference (GD) vs. Expected Goal Difference (xGD)',
        labels={
            'GD': 'ผลต่างประตูได้เสียจริง (GD)',
            'xGD': 'ผลต่างประตูได้เสียคาดหวัง (xGD)',
            'Performance Gap': 'ส่วนต่าง (GD - xGD)'
        },
        hover_data=['Rk', 'Pts', 'Performance Gap']
    )

    # เพิ่มเส้นแกนกลาง (GD = xGD) เพื่อแยก Over/Underperformance
    min_val = min(df['xGD'].min(), df['GD'].min()) - 2
    max_val = max(df['xGD'].max(), df['GD'].max()) + 2

    fig8.add_shape(
        type="line",
        x0=min_val,
        y0=min_val,
        x1=max_val,
        y1=max_val,
        line=dict(color="gray", width=1, dash="dash")
    )

    # เพิ่มคำอธิบาย Quadrant
    fig8.add_annotation(x=10, y=10, text="✨ YOD YIAM (Strong)", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
    fig8.add_annotation(x=10, y=-5, text="📉 UNDERPERFORMING", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
    fig8.add_annotation(x=-5, y=10, text="🍀 OVERPERFORMING", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
    fig8.add_annotation(x=-5, y=-5, text="❌ WEAK", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")


    fig8.update_traces(textposition='top center', mode='markers+text')
    fig8.update_layout(height=650)

    # คืนค่ากราฟทั้งหมด
    return fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8

# Main execution (แก้ไขส่วนนี้เพื่อแสดง fig8)
if __name__ == "__main__":
    print("="*70)
    print("⚽ PREMIER LEAGUE DATA ANALYSIS")
    print("="*70)

    # ใช้ URL ที่พยายามจะ Scraping แต่จะ Fallback ไปใช้ข้อมูลจากภาพ
    url = "https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691"

    print("\n🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n" + "="*70)
    print("🔍 KEY INSIGHTS - ข้อมูลเชิงลึกที่น่าสนใจ")
    print("="*70)

    insights = analyze_insights(df)
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")

    print("\n" + "="*70)
    print("📊 สถิติเพิ่มเติม")
    print("="*70)
    print(f"• ค่าเฉลี่ยประตูต่อเกมทั้งลีก: {df['Goals_per_Match'].mean():.2f}")
    print(f"• ทีมที่ทำประตูมากที่สุด 3 อันดับ: {', '.join(df.nlargest(3, 'GF')['Squad'].tolist())}")
    print(f"• ทีมที่เสียประตูน้อยที่สุด 3 อันดับ: {', '.join(df.nsmallest(3, 'GA')['Squad'].tolist())}")
    print(f"• ทีมที่มีผู้ชมมากที่สุด: {df.loc[df['Attendance'].idxmax(), 'Squad']} ({df['Attendance'].max():,} คน)")

    print("\n" + "="*70)
    print("📈 กำลังสร้างกราฟ...")
    print("="*70)

    # รับกราฟทั้งหมดรวมถึง fig8
    fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8 = create_visualizations(df)



    # แสดงกราฟอื่น ๆ (ตามโค้ดเดิม)
    print("\n**แสดงกราฟที่ 1**")
    fig1.show()
    print("\n**แสดงกราฟที่ 2**")
    fig2.show()
    print("\n**แสดงกราฟที่ 3**")
    fig3.show()
    print("\n**แสดงกราฟที่ 4**")
    fig4.show()
    print("\n**แสดงกราฟที่ 5**")
    fig5.show()
    print("\n**แสดงกราฟที่ 6**")
    fig6.show()
    print("\n**แสดงกราฟที่ 7**")
    fig7.show()

    # แสดงกราฟ GD vs xGD (กราฟหลักที่ต้องการ)
    print("\n**แสดงกราฟที่ 8 GD vs xGD (Performance)**")
    fig8.show()

    # แสดงตารางข้อมูล
    print("\n" + "="*70)
    print("📋 ตารางข้อมูลทั้งหมด")
    print("="*70)
    print(df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA', 'xGD', 'Performance Gap']].to_string(index=False))

    # บันทึกข้อมูล
    df.to_csv('premier_league_stats_2024_25.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ 'premier_league_stats_2024_25.csv' เรียบร้อยแล้ว!")

    # สรุปทีมที่น่าสนใจ
    print("\n" + "="*70)
    print("🌟 ทีมที่น่าจับตามอง")
    print("="*70)

    # ทีมที่ทำได้ดีกว่าที่คาด (Overperformers)
    overperformers = df[df['Performance Gap'] > 4].sort_values('Performance Gap', ascending=False)
    if not overperformers.empty:
        print("\n🚀 ทีมที่ทำได้เหนือความคาด (GD > xGD):")
        for _, team in overperformers.iterrows():
            print(f"   • {team['Squad']}: ส่วนต่าง +{team['Performance Gap']:.1f} (GD={team['GD']} vs xGD={team['xGD']:.1f})")

    # ทีมที่น่าเป็นห่วง (Underperformers)
    underperformers = df[df['Performance Gap'] < -4].sort_values('Performance Gap')
    if not underperformers.empty:
        print("\n⚠️ ทีมที่ยังทำได้ไม่ดีเท่าที่ควร (GD < xGD):")
        for _, team in underperformers.iterrows():
            print(f"   • {team['Squad']}: ส่วนต่าง {team['Performance Gap']:.1f} (GD={team['GD']} vs xGD={team['xGD']:.1f})")

⚽ PREMIER LEAGUE DATA ANALYSIS

🔄 กำลังดึงข้อมูล Premier League...
Found total table 24
    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13


**แสดงกราฟที่ 2**



**แสดงกราฟที่ 3**



**แสดงกราฟที่ 4**



**แสดงกราฟที่ 5**



**แสดงกราฟที่ 6**



**แสดงกราฟที่ 7**



**แสดงกราฟที่ 8 GD vs xGD (Performance)**



📋 ตารางข้อมูลทั้งหมด
 Rk             Squad  MP  W  D  L  GF  GA  GD  Pts   xG  xGA  xGD  Performance Gap
  1           Arsenal   9  7  1  1  16   3  13   22 14.6  5.2  9.4              3.6
  2       Bournemouth   9  5  3  1  16  11   5   18 11.0 11.1 10.0             -5.0
  3         Tottenham   9  5  2  2  17   7  10   17 10.3 10.5 11.3             -1.3
  4        Sunderland   9  5  2  2  11   7   4   17  8.6 10.4  8.1             -4.1
  5   Manchester City   9  5  1  3  17   7  10   16 15.6  9.0  5.9              4.1
  6    Manchester Utd   9  5  1  3  15  14   1   16 16.2 13.6 14.6            -13.6
  7         Liverpool   9  5  0  4  16  14   2   15 15.8 12.0  3.8             -1.8
  8       Aston Villa   9  4  3  2   9   8   1   15  7.1 11.8  0.3              0.7
  9           Chelsea   9  4  2  3  17  11   6   14 13.4 13.1  5.3              0.7
 10    Crystal Palace   9  3  4  2  12   9   3   13 17.6 11.7 17.6            -14.6
 11         Brentford   9  4  1  4  14  14   0   13 13

In [15]:
import pandas as pd
import plotly.express as px

# 1. สร้าง DataFrame จากข้อมูลในรูปภาพ
data = {
    'Rk': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Squad': ['Arsenal', 'Bournemouth', 'Tottenham', 'Sunderland', 'Manchester City',
              'Manchester Utd', 'Liverpool', 'Aston Villa', 'Chelsea', 'Crystal Palace',
              'Brentford', 'Newcastle Utd', 'Brighton', 'Everton', 'Leeds United',
              'Burnley', 'Fulham', 'Nottm Forest', 'West Ham', 'Wolves'],
    'GD': [16, 11, 10, 7, 6, 1, 2, 1, 3, 3, 0, 0, -1, 3, 0, -5, -5, -5, -7, -7],
    'Pts': [24, 20, 19, 18, 17, 16, 16, 15, 14, 13, 13, 12, 12, 12, 11, 10, 9, 8, 4, 2],
    'xGD': [9.3, 11.1, 10.5, 9.0, 9.0, 13.6, 12.0, 7.1, 13.1, 17.6, 13.4, 7.0, 13.3, 12.3, 9.0, 6.8, 9.1, 7.6, 7.6, 9.2],
    'GF': [26, 18, 17, 12, 15, 15, 16, 9, 11, 11, 14, 9, 12, 12, 9, 12, 5, 5, 7, 9],
    'xG': [2.44 * 9, 2.00 * 9, 1.89 * 9, 1.89 * 9, 1.78 * 9, 1.78 * 9, 1.78 * 9, 1.67 * 9, 1.56 * 9, 1.44 * 9, 1.44 * 9, 1.33 * 9, 1.33 * 9, 1.22 * 9, 1.22 * 9, 1.11 * 9, 1.00 * 9, 0.56 * 9, 0.44 * 9, 0.22 * 9],
    'xGD/90': [1.04, 1.11, 1.26, 0.90, 0.66, 1.62, 0.42, 0.03, 0.59, 1.96, 1.49, 0.78, 1.48, 1.37, 1.00, 0.76, -0.42, -0.80, -0.89, -0.25], # ใช้ค่า xGD/90 จากรูปภาพ
    # เนื่องจากค่า xG/90 และ xGA/90 ถูกรวมในคอลัมน์ Pts/Mp และ xG/xGA/xGD เราจะใช้ xGD/90 ที่ชัดเจนที่สุด
}

# แปลงค่า xGD/90 ให้เป็น xGD รวม (xGD = xGD/90 * MP)
# ในข้อมูลนี้ MP คือ 9 นัด (ยกเว้นบางทีมที่อาจจะ 7) แต่ส่วนใหญ่เป็น 9
df = pd.DataFrame(data)
df['xGD'] = df['xGD/90'] * 9

# 2. คำนวณความแตกต่างเพื่อหา Over/Under Performance
df['Performance Gap'] = df['GD'] - df['xGD']

# 3. สร้าง Scatter Plot ด้วย Plotly
fig = px.scatter(
    df,
    x='xGD',
    y='GD',
    text='Squad',
    size='Pts', # ขนาดวงกลมตามคะแนน (Pts) เพื่อแสดงอันดับ
    color='Performance Gap', # สีตาม Performance Gap
    color_continuous_scale=px.colors.diverging.RdYlGn, # ใช้โทนสีเขียว-เหลือง-แดง
    title='⚽ ประสิทธิภาพพรีเมียร์ลีก: ผลต่างประตูจริง (GD) vs. ผลต่างประตูคาดหวัง (xGD)',
    labels={
        'GD': 'ผลต่างประตูได้เสียจริง (GD)',
        'xGD': 'ผลต่างประตูได้เสียคาดหวัง (xGD)'
    },
    hover_data=['Rk', 'Pts', 'Performance Gap']
)

# เพิ่มเส้นแกนกลาง (GD = xGD)
fig.add_shape(
    type="line",
    x0=df['xGD'].min() - 1,
    y0=df['xGD'].min() - 1,
    x1=df['xGD'].max() + 1,
    y1=df['xGD'].max() + 1,
    line=dict(color="LightSlateGray", width=1, dash="dash")
)

# เพิ่มคำอธิบาย Quadrant
fig.add_annotation(x=12, y=10, text="A: YOD YIAM (Strong)", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
fig.add_annotation(x=12, y=0, text="B: UNLUCKY / POOR FINISHING", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
fig.add_annotation(x=6, y=10, text="C: LUCKY / SUPER EFFICIENCY", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")
fig.add_annotation(x=6, y=0, text="D: WEAK PERFORMANCE", showarrow=False, bgcolor="rgba(255, 255, 255, 0.5)")


fig.update_traces(textposition='top center')
fig.show()

# พิมพ์ตารางผลต่าง
print("\n--- Performance Gap (GD - xGD) ---")
print(df[['Squad', 'GD', 'xGD', 'Performance Gap']].sort_values(by='Performance Gap', ascending=False).to_string(index=False))


--- Performance Gap (GD - xGD) ---
          Squad  GD   xGD  Performance Gap
        Arsenal  16  9.36             6.64
   Nottm Forest  -5 -7.20             2.20
       West Ham  -7 -8.01             1.01
    Bournemouth  11  9.99             1.01
    Aston Villa   1  0.27             0.73
Manchester City   6  5.94             0.06
     Sunderland   7  8.10            -1.10
         Fulham  -5 -3.78            -1.22
      Tottenham  10 11.34            -1.34
      Liverpool   2  3.78            -1.78
        Chelsea   3  5.31            -2.31
         Wolves  -7 -2.25            -4.75
  Newcastle Utd   0  7.02            -7.02
   Leeds United   0  9.00            -9.00
        Everton   3 12.33            -9.33
        Burnley  -5  6.84           -11.84
      Brentford   0 13.41           -13.41
 Manchester Utd   1 14.58           -13.58
       Brighton  -1 13.32           -14.32
 Crystal Palace   3 17.64           -14.64


In [14]:
# Homework 2
# DADS5001 : Tools and Programing (2nd Half)
# Siriwat Carawukh 6720422017

import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import re

# Scrape data from Web
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    try:
        # Read table from web url
        tables = pd.read_html(url)
        # Check number of tables
        print(f"Found total table {len(tables)}")
        # Define table[0] for Premier League Table Statistic
        df = tables[0]
        # Show dataframe
        print(df)

        if df :
            print(f"✅ ดึงข้อมูลสำเร็จ! พบ {len(df)} ทีม")
            print(f"📊 Columns: {df.columns.tolist()}")
            return df

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        print("💡 ใช้ข้อมูลจากภาพที่แนบมา")

    return create_data_from_image()

def create_data_from_image():
    """
    สร้างข้อมูลจากภาพที่แนบมา (ข้อมูลจริง Premier League 2024-25)
    """
    data = {
        'Rk': list(range(1, 21)),
        'Squad': ['Arsenal', 'Bournemouth', 'Tottenham', 'Sunderland', 'Manchester City',
                 'Manchester Utd', 'Liverpool', 'Aston Villa', 'Chelsea', 'Crystal Palace',
                 'Brentford', 'Newcastle Utd', 'Brighton', 'Everton', 'Leeds United',
                 'Burnley', 'Fulham', 'Nottingham Forest', 'West Ham', 'Wolves'],
        'MP': [9] * 20,
        'W': [7, 5, 5, 5, 5, 5, 5, 4, 4, 3, 4, 3, 3, 3, 3, 3, 2, 1, 1, 0],
        'D': [1, 3, 2, 2, 1, 1, 0, 3, 2, 4, 1, 3, 3, 2, 2, 1, 2, 2, 1, 2],
        'L': [1, 1, 2, 2, 3, 3, 4, 2, 3, 2, 4, 3, 3, 4, 4, 5, 5, 6, 7, 7],
        'GF': [16, 16, 17, 11, 17, 15, 16, 9, 17, 12, 14, 9, 14, 9, 9, 12, 9, 5, 7, 7],
        'GA': [3, 11, 7, 7, 7, 14, 14, 8, 11, 9, 14, 8, 15, 12, 14, 17, 14, 17, 20, 19],
        'GD': [13, 5, 10, 4, 10, 1, 2, 1, 6, 3, 0, 1, -1, -3, -5, -5, -5, -12, -13, -12],
        'Pts': [22, 18, 17, 17, 16, 16, 15, 15, 14, 13, 13, 12, 12, 11, 11, 10, 8, 5, 4, 2],
        'Pts/MP': [2.44, 2.00, 1.89, 1.89, 1.78, 1.78, 1.67, 1.67, 1.56, 1.44,
                  1.44, 1.33, 1.33, 1.22, 1.22, 1.11, 0.89, 0.56, 0.44, 0.22],
        'xG': [14.6, 11.0, 10.3, 8.6, 15.6, 16.2, 15.8, 7.1, 13.4, 17.6,
              13.4, 11.9, 13.2, 12.3, 11.5, 6.8, 9.0, 9.1, 7.6, 9.2],
        'xGA': [5.2, 11.1, 10.5, 10.4, 9.0, 13.6, 12.0, 11.8, 13.1, 11.7,
               12.2, 7.0, 11.8, 14.0, 9.0, 18.1, 12.8, 14.7, 14.8, 11.4],
        'xGD': [9.3, -0.1, -0.2, -1.8, 6.6, 2.6, 3.8, -4.7, 0.3, 5.9,
               1.2, 4.9, 1.4, -1.7, 2.5, -11.3, -3.8, -5.5, -7.2, -2.2],
        'Attendance': [60144, 11164, 61039, 46335, 52311, 73965, 60362, 41242,
                      39640, 25068, 17014, 52183, 31518, 51883, 36722, 21521,
                      27172, 30221, 62457, 30231],
        'Top_Scorer': ['Viktor Gyökeres - 3', 'Antoine Semenyo - 6', 'Richarlison, Nicky van de Ven - 3',
                      'Wilson Isidor - 4', 'Erling Haaland - 11', 'Bryan Mbeumo - 4',
                      'Mohamed Salah, Cody Gakpo - 3', 'Emi Buendía, Donyell Malen - 2',
                      'Enzo Fernández, Moisés Caicedo - 3', 'Jean-Philippe Mateta - 5',
                      'Thiago - 6', 'Nick Woltemade - 4', 'Danny Welbeck - 5',
                      'Iliman Ndiaye - 3', 'Joe Rodon, Noah Okafor - 2', 'Jaidon Anthony - 4',
                      'Raúl Jiménez, Alex Iwobi - 1', 'Chris Wood - 2', 'Jarrod Bowen - 3',
                      'Hwang Hee-chan, Santiago Bueno - 1'],
        'Goalkeeper' : ['David Raya', 'Đorđe Petrović', 'Guglielmo Vicario', 'Robin Roefs', 'Gianluigi Donnarumma',
                        'Altay Bayındır', 'Alisson', 'Emiliano Martínez', 'Robert Sánchez', 'Dean Henderson',
                        'Caoimhín Kelleher', 'Nick Pope', 'Bart Verbruggen', 'Jordan Pickford','Karl Darlow',
                        'Martin Dúbravka', 'Bernd Leno', 'Matz Sels', 'Alphonse Areola', 'Sam Johnstone']
    }

    df = pd.DataFrame(data)

    # คำนวณค่าเพิ่มเติม
    df['Win_Rate'] = (df['W'] / df['MP'] * 100).round(1)
    df['xG_diff'] = (df['GF'] - df['xG']).round(1)
    df['xGA_diff'] = (df['GA'] - df['xGA']).round(1)
    df['Goals_per_Match'] = (df['GF'] / df['MP']).round(2)
    df['Conceded_per_Match'] = (df['GA'] / df['MP']).round(2)

    return df

def analyze_insights(df):
    """
    วิเคราะห์หา insights ที่น่าสนใจจากข้อมูล
    """
    insights = []

    # 1. ทีมที่นำตาราง
    top_team = df.iloc[0]
    insights.append(f"🏆 ทีมอันดับ 1: {top_team['Squad']} ({top_team['Pts']} คะแนน, {top_team['W']} ชนะ)")

    # 2. ทีมที่ทำประตูได้มากที่สุด
    top_scorer_team = df.loc[df['GF'].idxmax()]
    insights.append(f"⚽ ทำประตูมากสุด: {top_scorer_team['Squad']} ({top_scorer_team['GF']} ประตู)")

    # 3. ทีมที่รับประตูน้อยที่สุด
    best_defense = df.loc[df['GA'].idxmin()]
    insights.append(f"🛡️ ป้องกันดีสุด: {best_defense['Squad']} (เสีย {best_defense['GA']} ประตูเท่านั้น)")

    # 4. ทีมที่มี xG diff สูงสุด (โชคดี/ทำได้เกินคาด)
    lucky_team = df.loc[df['xG_diff'].idxmax()]
    if lucky_team['xG_diff'] > 2:
        insights.append(f"⚡ ทำประตูเกินคาด: {lucky_team['Squad']} (+{lucky_team['xG_diff']:.1f} เทียบกับ xG)")

    # 5. ทีมที่มี xGA diff ติดลบมาก (โชคดีในการป้องกัน)
    lucky_defense = df.loc[df['xGA_diff'].idxmin()]
    if lucky_defense['xGA_diff'] < -2:
        insights.append(f"🍀 เสียประตูน้อยกว่าที่คาด: {lucky_defense['Squad']} ({lucky_defense['xGA_diff']:.1f} เทียบกับ xGA)")

    # 6. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.loc[df['GD'].idxmax()]
    insights.append(f"📊 ผลต่างประตูดีสุด: {best_gd['Squad']} (+{best_gd['GD']})")

    # 7. ทีมที่มีผู้ชมเยอะที่สุด
    most_attendance = df.loc[df['Attendance'].idxmax()]
    insights.append(f"👥 ผู้ชมเฉลี่ยสูงสุด: {most_attendance['Squad']} ({most_attendance['Attendance']:,} คน)")

    # 8. ทีมที่มีอัตราชนะสูงสุด
    best_win_rate = df.loc[df['Win_Rate'].idxmax()]
    insights.append(f"📈 อัตราชนะสูงสุด: {best_win_rate['Squad']} ({best_win_rate['Win_Rate']:.1f}%)")

    # 9. ทีมที่ดิ้นรนหนัก (xGD ดีแต่คะแนนน้อย)
    df['xPts_estimate'] = (df['W'] * 3 + df['D'])  # คะแนนจริง
    unlucky_teams = df[df['xGD'] > 0].nsmallest(3, 'Pts')
    if not unlucky_teams.empty:
        insights.append(f"😓 ทีมที่น่าจะได้คะแนนมากกว่านี้: {', '.join(unlucky_teams['Squad'].tolist())}")

    return insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ที่หลากหลาย
    """

    # 1. ตารางคะแนน Top 10
    fig1 = go.Figure()

    top10 = df.head(10).sort_values('Pts', ascending=True)

    fig1.add_trace(go.Bar(
        y=top10['Squad'],
        x=top10['Pts'],
        orientation='h',
        marker=dict(
            color=top10['Pts'],
            colorscale='Viridis',
            showscale=True
        ),
        text=top10['Pts'],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>คะแนน: %{x}<br><extra></extra>'
    ))

    fig1.update_layout(
        title='🏆 Premier League - Top 10 Teams',
        xaxis_title='คะแนน (Points)',
        yaxis_title='ทีม',
        height=600,
        template='plotly_white'
    )

    # 2. Goals Scored vs Conceded (Bubble Chart)
    fig2 = px.scatter(df,
                      x='GF',
                      y='GA',
                      size='Pts',
                      color='Pts',
                      hover_name='Squad',
                      size_max=50,
                      color_continuous_scale='RdYlGn',
                      title='⚽ Goals Scored vs Goals Conceded (ขนาดวง = คะแนน)')

    fig2.add_shape(type='line', x0=0, y0=0, x1=25, y1=25,
                   line=dict(color='gray', dash='dash'),
                   name='เสมอ')

    fig2.update_layout(
        xaxis_title='ประตูที่ทำได้ (GF)',
        yaxis_title='ประตูที่เสีย (GA)',
        height=600
    )

    # 3. Actual Goals vs Expected Goals (xG)
    fig3 = go.Figure()

    # เรียงตามคะแนน
    df_sorted = df.sort_values('Pts', ascending=True)

    fig3.add_trace(go.Bar(
        name='Actual Goals (GF)',
        x=df_sorted['Squad'],
        y=df_sorted['GF'],
        marker_color='steelblue'
    ))

    fig3.add_trace(go.Scatter(
        name='Expected Goals (xG)',
        x=df_sorted['Squad'],
        y=df_sorted['xG'],
        mode='markers',
        marker=dict(size=10, color='red', symbol='diamond')
    ))

    fig3.update_layout(
        title='📈 Actual Goals vs Expected Goals (xG)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนประตู',
        xaxis_tickangle=-45,
        height=600,
        template='plotly_white',
        barmode='group'
    )

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    df_sorted = df.sort_values('Pts', ascending=False)

    fig4.add_trace(go.Bar(
        name='ชนะ (W)',
        x=df_sorted['Squad'],
        y=df_sorted['W'],
        marker_color='green'
    ))

    fig4.add_trace(go.Bar(
        name='เสมอ (D)',
        x=df_sorted['Squad'],
        y=df_sorted['D'],
        marker_color='orange'
    ))

    fig4.add_trace(go.Bar(
        name='แพ้ (L)',
        x=df_sorted['Squad'],
        y=df_sorted['L'],
        marker_color='red'
    ))

    fig4.update_layout(
        title='📊 การกระจายผลการแข่งขัน (W/D/L)',
        xaxis_title='ทีม',
        yaxis_title='จำนวนเกม',
        barmode='stack',
        xaxis_tickangle=-45,
        height=600
    )

    # 5. xG Difference (Over/Under Performance)
    fig5 = go.Figure()

    df_sorted = df.sort_values('xG_diff', ascending=True)

    colors = ['red' if x < 0 else 'green' for x in df_sorted['xG_diff']]

    fig5.add_trace(go.Bar(
        x=df_sorted['Squad'],
        y=df_sorted['xG_diff'],
        marker_color=colors,
        text=df_sorted['xG_diff'].round(1),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>xG Diff: %{y:.1f}<extra></extra>'
    ))

    fig5.add_hline(y=0, line_dash="dash", line_color="black")

    fig5.update_layout(
        title='🎯 Actual Goals - Expected Goals (มากกว่า xG = สีเขียว, น้อยกว่า = สีแดง)',
        xaxis_title='ทีม',
        yaxis_title='ผลต่าง (GF - xG)',
        xaxis_tickangle=-45,
        height=600
    )

    # 6. Attendance vs Performance
    fig6 = px.scatter(df,
                     x='Attendance',
                     y='Pts',
                     size='GF',
                     color='GD',
                     hover_name='Squad',
                     title='👥 ผู้ชม vs ผลงาน (ขนาด = จำนวนประตูที่ทำได้)',
                     color_continuous_scale='RdYlGn')

    fig6.update_layout(
        xaxis_title='จำนวนผู้ชมเฉลี่ย',
        yaxis_title='คะแนน',
        height=600
    )

    # 7. Goal Difference Ranking
    fig7 = px.bar(df.sort_values('GD', ascending=True),
                  x='GD',
                  y='Squad',
                  orientation='h',
                  color='GD',
                  color_continuous_scale='RdYlGn',
                  title='🔢 Goal Difference (ผลต่างประตู)')

    fig7.update_layout(height=700)

    return fig1, fig2, fig3, fig4, fig5, fig6, fig7

# Main execution
if __name__ == "__main__":
    print("="*70)
    print("⚽ PREMIER LEAGUE DATA ANALYSIS")
    print("="*70)

    url = "https://fbref.com/en/comps/9/Premier-League-Stats"

    print("\n🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n" + "="*70)
    print("🔍 KEY INSIGHTS - ข้อมูลเชิงลึกที่น่าสนใจ")
    print("="*70)

    insights = analyze_insights(df)
    for i, insight in enumerate(insights, 1):
        print(f"{i}. {insight}")

    print("\n" + "="*70)
    print("📊 สถิติเพิ่มเติม")
    print("="*70)
    print(f"• ค่าเฉลี่ยประตูต่อเกมทั้งลีก: {df['Goals_per_Match'].mean():.2f}")
    print(f"• ทีมที่ทำประตูมากที่สุด 3 อันดับ: {', '.join(df.nlargest(3, 'GF')['Squad'].tolist())}")
    print(f"• ทีมที่เสียประตูน้อยที่สุด 3 อันดับ: {', '.join(df.nsmallest(3, 'GA')['Squad'].tolist())}")
    print(f"• ทีมที่มีผู้ชมมากที่สุด: {df.loc[df['Attendance'].idxmax(), 'Squad']} ({df['Attendance'].max():,} คน)")

    print("\n" + "="*70)
    print("📈 กำลังสร้างกราฟ...")
    print("="*70)


    # Initialize figure with subplots
    fig = make_subplots(
    rows=3, cols=3,
    column_widths=[0.33, 0.33, 0.33],
    row_heights=[0.33, 0.33, 0.33],
    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}, {"type": "bar"}],
           [            None                    , {"type": "surface"}, {"type": "surface"}],
           [            None                    , {"type": "surface"}, {"type": "surface"}]])


    fig1, fig2, fig3, fig4, fig5, fig6, fig7 = create_visualizations(df)

    # Show all plots
    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()
    fig5.show()
    fig6.show()
    fig7.show()

    # แสดงตารางข้อมูล
    print("\n" + "="*70)
    print("📋 ตารางข้อมูลทั้งหมด")
    print("="*70)
    print(df[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'xG', 'xGA']].to_string(index=False))

    # บันทึกข้อมูล
    df.to_csv('premier_league_stats_2024_25.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ 'premier_league_stats_2024_25.csv' เรียบร้อยแล้ว!")

    # สรุปทีมที่น่าสนใจ
    print("\n" + "="*70)
    print("🌟 ทีมที่น่าจับตามอง")
    print("="*70)

    # ทีมที่ทำได้ดีกว่าที่คาด
    overperformers = df[df['xG_diff'] > 2].sort_values('xG_diff', ascending=False)
    if not overperformers.empty:
        print("\n🚀 ทีมที่ทำได้เหนือความคาด (xG):")
        for _, team in overperformers.iterrows():
            print(f"   • {team['Squad']}: ทำได้ {team['GF']} ประตู (คาดหวัง {team['xG']:.1f})")

    # ทีมที่น่าเป็นห่วง
    underperformers = df[df['xG_diff'] < -2].sort_values('xG_diff')
    if not underperformers.empty:
        print("\n⚠️ ทีมที่ยังทำได้ไม่ดีเท่าที่ควร:")
        for _, team in underperformers.iterrows():
            print(f"   • {team['Squad']}: ทำได้ {team['GF']} ประตู (คาดหวัง {team['xG']:.1f})")

⚽ PREMIER LEAGUE DATA ANALYSIS

🔄 กำลังดึงข้อมูล Premier League...
Found total table 24
    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13


📋 ตารางข้อมูลทั้งหมด
 Rk             Squad  MP  W  D  L  GF  GA  GD  Pts   xG  xGA
  1           Arsenal   9  7  1  1  16   3  13   22 14.6  5.2
  2       Bournemouth   9  5  3  1  16  11   5   18 11.0 11.1
  3         Tottenham   9  5  2  2  17   7  10   17 10.3 10.5
  4        Sunderland   9  5  2  2  11   7   4   17  8.6 10.4
  5   Manchester City   9  5  1  3  17   7  10   16 15.6  9.0
  6    Manchester Utd   9  5  1  3  15  14   1   16 16.2 13.6
  7         Liverpool   9  5  0  4  16  14   2   15 15.8 12.0
  8       Aston Villa   9  4  3  2   9   8   1   15  7.1 11.8
  9           Chelsea   9  4  2  3  17  11   6   14 13.4 13.1
 10    Crystal Palace   9  3  4  2  12   9   3   13 17.6 11.7
 11         Brentford   9  4  1  4  14  14   0   13 13.4 12.2
 12     Newcastle Utd   9  3  3  3   9   8   1   12 11.9  7.0
 13          Brighton   9  3  3  3  14  15  -1   12 13.2 11.8
 14           Everton   9  3  2  4   9  12  -3   11 12.3 14.0
 15      Leeds United   9  3  2  4   9  14  -5  

In [34]:
# Web scrape example

import pandas as pd
# URL ของตาราง
url = "https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691"
# อ่านตารางทั้งหมดจากหน้าเว็บ
tables = pd.read_html(url)
# ตรวจสอบว่ามีกี่ตาราง
print(f"พบทั้งหมด {len(tables)} ตาราง")
# ลองดูตารางที่น่าจะเป็น Premier League Table (มักจะเป็นอันแรก)
df = tables[0] # แสดงตัวอย่างข้อมูล
print(df)

พบทั้งหมด 24 ตาราง
    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
11  12    Newcastle Utd   9  3  3  3   9   8   1   12    

In [ ]:
!pip install pandas requests beautifulsoup4 plotly lxml html5lib selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 26.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ฟังก์ชันสำหรับดึงข้อมูลจาก FBref
def scrape_premier_league_data(url):
    """
    Scrape Premier League statistics from FBref
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 2. Fetch the HTML content
    try:
      response = requests.get(url, headers=headers)
      response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
      html_content = response.text
    except requests.exceptions.RequestException as e:
      print(f"Error fetching the URL: {e}")
    # Continue execution even if there's an error, although parsing will fail
    html_content = None

    # Correct table by inspect
    # หาตารางสถิติทีม (League Table)
    table = soup.find('table', {'id': 'results2025-202691_overall'})

    if table:
        # อ่านตารางด้วย pandas
        df = pd.read_html(str(table))[0]

        # ทำความสะอาดข้อมูล
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = ['_'.join(col).strip() if col[1] else col[0]
                         for col in df.columns.values]

        return df
    else:
        print("ไม่พบตาราง ลองใช้ข้อมูล demo")
        return create_demo_data()


# Just a DEMO ! : This is not necessary.
def create_demo_data():
    """
    สร้างข้อมูล demo สำหรับการวิเคราะห์
    """
    data = {
        'Squad': ['Liverpool', 'Arsenal', 'Man City', 'Chelsea', 'Aston Villa',
                 'Brighton', 'Newcastle', 'Tottenham', 'Man United', 'West Ham',
                 'Fulham', 'Brentford', 'Wolves', 'Nottm Forest', 'Bournemouth',
                 'Everton', 'Leicester', 'Crystal Palace', 'Ipswich', 'Southampton'],
        'MP': [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        'W': [8, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 1, 1],
        'D': [2, 2, 2, 3, 3, 3, 3, 1, 3, 2, 2, 2, 4, 4, 3, 2, 3, 2, 3, 1],
        'L': [0, 1, 1, 1, 1, 2, 2, 4, 3, 4, 4, 4, 3, 3, 4, 5, 5, 6, 6, 8],
        'GF': [19, 18, 20, 20, 17, 16, 11, 18, 11, 13, 13, 18, 14, 11, 12, 10, 12, 8, 9, 7],
        'GA': [4, 8, 9, 11, 11, 12, 10, 10, 10, 16, 12, 18, 18, 13, 16, 15, 18, 12, 18, 18],
        'GD': [15, 10, 11, 9, 6, 4, 1, 8, 1, -3, 1, 0, -4, -2, -4, -5, -6, -4, -9, -11],
        'Pts': [26, 23, 23, 21, 21, 18, 18, 16, 15, 14, 14, 14, 13, 13, 12, 11, 9, 8, 6, 4],
        'xG': [20.5, 19.2, 22.1, 18.5, 16.8, 15.3, 12.4, 19.8, 12.2, 14.1, 13.8, 17.2, 13.5, 10.8, 11.9, 9.7, 11.5, 9.2, 8.8, 8.1],
        'xGA': [7.2, 9.5, 10.2, 12.3, 11.8, 13.1, 11.5, 12.8, 11.9, 15.8, 13.2, 16.5, 16.8, 12.9, 14.7, 14.2, 17.3, 13.5, 17.2, 19.5]
    }
    return pd.DataFrame(data)

def analyze_data(df):
    """
    วิเคราะห์ข้อมูลและสร้าง insights
    """
    insights = []

    # 1. ทีมที่มีประสิทธิภาพการทำประตูดีที่สุด
    df['Goals_per_Match'] = df['GF'] / df['MP']
    df['Conceded_per_Match'] = df['GA'] / df['MP']

    top_scorer = df.nlargest(1, 'GF')['Squad'].values[0]
    insights.append(f"🎯 ทีมที่ทำประตูได้มากที่สุด: {top_scorer} ({df[df['Squad']==top_scorer]['GF'].values[0]} ประตู)")

    # 2. ทีมที่มีการป้องกันดีที่สุด
    best_defense = df.nsmallest(1, 'GA')['Squad'].values[0]
    insights.append(f"🛡️ ทีมที่เสียประตูน้อยที่สุด: {best_defense} ({df[df['Squad']==best_defense]['GA'].values[0]} ประตู)")

    # 3. ทีมที่มี Goal Difference ดีที่สุด
    best_gd = df.nlargest(1, 'GD')['Squad'].values[0]
    insights.append(f"📊 ผลต่างประตูดีที่สุด: {best_gd} (+{df[df['Squad']==best_gd]['GD'].values[0]})")

    # 4. วิเคราะห์ xG (Expected Goals)
    if 'xG' in df.columns and 'xGA' in df.columns:
        df['xG_diff'] = df['GF'] - df['xG']
        df['xGA_diff'] = df['GA'] - df['xGA']

        overperformer = df.nlargest(1, 'xG_diff')['Squad'].values[0]
        insights.append(f"⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): {overperformer}")

        lucky_defense = df.nsmallest(1, 'xGA_diff')['Squad'].values[0]
        insights.append(f"🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): {lucky_defense}")

    # 5. อัตราชนะ
    df['Win_Rate'] = (df['W'] / df['MP']) * 100

    # 6. คะแนนต่อเกม
    df['Points_per_Game'] = df['Pts'] / df['MP']

    return df, insights

def create_visualizations(df):
    """
    สร้างกราฟวิเคราะห์ต่างๆ
    """

    # 1. Top 10 Teams by Points
    fig1 = px.bar(df.nlargest(10, 'Pts').sort_values('Pts', ascending=True),
                  x='Pts', y='Squad', orientation='h',
                  title='🏆 Top 10 Teams - Total Points',
                  labels={'Pts': 'Points', 'Squad': 'Team'},
                  color='Pts',
                  color_continuous_scale='Viridis')
    fig1.update_layout(height=500, showlegend=False)

    # 2. Goals Scored vs Conceded
    fig2 = px.scatter(df, x='GF', y='GA', text='Squad',
                      size='Pts', color='Pts',
                      title='⚽ Goals Scored vs Goals Conceded',
                      labels={'GF': 'Goals For', 'GA': 'Goals Against'},
                      color_continuous_scale='RdYlGn_r')
    fig2.update_traces(textposition='top center')
    fig2.update_layout(height=600)

    # 3. Expected Goals Analysis (ถ้ามีข้อมูล xG)
    if 'xG' in df.columns and 'xGA' in df.columns:
        fig3 = go.Figure()

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['xG'],
            name='Expected Goals (xG)',
            mode='markers',
            marker=dict(size=10, color='lightblue')
        ))

        fig3.add_trace(go.Scatter(
            x=df['Squad'], y=df['GF'],
            name='Actual Goals',
            mode='markers',
            marker=dict(size=10, color='darkblue')
        ))

        fig3.update_layout(
            title='📈 Actual Goals vs Expected Goals (xG)',
            xaxis_title='Team',
            yaxis_title='Goals',
            height=600,
            xaxis_tickangle=-45
        )
    else:
        fig3 = None

    # 4. Win/Draw/Loss Distribution
    fig4 = go.Figure()

    fig4.add_trace(go.Bar(name='Wins', x=df['Squad'], y=df['W'], marker_color='green'))
    fig4.add_trace(go.Bar(name='Draws', x=df['Squad'], y=df['D'], marker_color='orange'))
    fig4.add_trace(go.Bar(name='Losses', x=df['Squad'], y=df['L'], marker_color='red'))

    fig4.update_layout(
        title='📊 Match Results Distribution',
        xaxis_title='Team',
        yaxis_title='Number of Matches',
        barmode='stack',
        height=600,
        xaxis_tickangle=-45
    )

    # 5. Points per Game
    fig5 = px.bar(df.sort_values('Points_per_Game', ascending=False),
                  x='Squad', y='Points_per_Game',
                  title='💯 Points per Game Average',
                  labels={'Points_per_Game': 'Points/Game', 'Squad': 'Team'},
                  color='Points_per_Game',
                  color_continuous_scale='Blues')
    fig5.update_layout(height=600, xaxis_tickangle=-45)

    return fig1, fig2, fig3, fig4, fig5

# Main execution
if __name__ == "__main__":
    url = "https://fbref.com/en/comps/9/Premier-League-Stats"

    print("🔄 กำลังดึงข้อมูล Premier League...")
    df = scrape_premier_league_data(url)

    print("\n📊 กำลังวิเคราะห์ข้อมูล...")
    df, insights = analyze_data(df)

    print("\n" + "="*60)
    print("🔍 INSIGHTS - ข้อมูลที่น่าสนใจ")
    print("="*60)
    for insight in insights:
        print(insight)

    print("\n" + "="*60)
    print("📈 สร้างกราฟวิเคราะห์...")
    print("="*60)

    fig1, fig2, fig3, fig4, fig5 = create_visualizations(df)

    # แสดงกราฟ
    fig1.show()
    fig2.show()
    if fig3:
        fig3.show()
    fig4.show()
    fig5.show()

    # แสดง DataFrame
    print("\n📋 ตารางสถิติทั้งหมด:")
    print(df.to_string())

    df.head(5)

    # บันทึกข้อมูลเป็น CSV (optional)
    df.to_csv('premier_league_stats.csv', index=False, encoding='utf-8-sig')
    print("\n✅ บันทึกข้อมูลเป็นไฟล์ premier_league_stats.csv เรียบร้อยแล้ว!")

🔄 กำลังดึงข้อมูล Premier League...
Error fetching the URL: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/9/Premier-League-Stats
ไม่พบตาราง ลองใช้ข้อมูล demo

📊 กำลังวิเคราะห์ข้อมูล...

🔍 INSIGHTS - ข้อมูลที่น่าสนใจ
🎯 ทีมที่ทำประตูได้มากที่สุด: Man City (20 ประตู)
🛡️ ทีมที่เสียประตูน้อยที่สุด: Liverpool (4 ประตู)
📊 ผลต่างประตูดีที่สุด: Liverpool (+15)
⚡ ทีมที่ทำประตูได้มากกว่าที่คาด (xG): Chelsea
🍀 ทีมที่เสียประตูน้อยกว่าที่คาด (xGA): Liverpool

📈 สร้างกราฟวิเคราะห์...



📋 ตารางสถิติทั้งหมด:
             Squad  MP  W  D  L  GF  GA  GD  Pts    xG   xGA  Goals_per_Match  Conceded_per_Match  xG_diff  xGA_diff  Win_Rate  Points_per_Game
0        Liverpool  10  8  2  0  19   4  15   26  20.5   7.2              1.9                 0.4     -1.5      -3.2      80.0              2.6
1          Arsenal  10  7  2  1  18   8  10   23  19.2   9.5              1.8                 0.8     -1.2      -1.5      70.0              2.3
2         Man City  10  7  2  1  20   9  11   23  22.1  10.2              2.0                 0.9     -2.1      -1.2      70.0              2.3
3          Chelsea  10  6  3  1  20  11   9   21  18.5  12.3              2.0                 1.1      1.5      -1.3      60.0              2.1
4      Aston Villa  10  6  3  1  17  11   6   21  16.8  11.8              1.7                 1.1      0.2      -0.8      60.0              2.1
5         Brighton  10  5  3  2  16  12   4   18  15.3  13.1              1.6                 1.2      0.7      -1

# Web Scrape data

In [11]:
# Simple web scraping : NOT work
import pandas as pd
url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

# table : results2025-202691_overall
#df = pd.read_html(url, attrs={"id":"results2025-202691_overall"})[0]
df = pd.read_html(url)[0]
df

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,Arsenal,9,7,1,1,16,3,13,22,2.44,14.6,5.2,9.3,1.04,D W W W W,60144,Viktor Gyökeres - 3,David Raya,NaN
1,2,Bournemouth,9,5,3,1,16,11,5,18,2.00,11.0,11.1,-0.1,-0.01,D D W D W,11164,Antoine Semenyo - 6,Đorđe Petrović,NaN
2,3,Tottenham,9,5,2,2,17,7,10,17,1.89,9.9,10.7,-0.8,-0.09,D D W L W,61039,"Richarlison, Micky van de Ven - 3",Guglielmo Vicario,NaN
3,4,Sunderland,9,5,2,2,11,7,4,17,1.89,8.6,10.4,-1.8,-0.20,D W L W W,46335,Wilson Isidor - 4,Robin Roefs,NaN
4,5,Manchester City,9,5,1,3,17,7,10,16,1.78,15.6,9.0,6.6,0.74,D W W W L,52311,Erling Haaland - 11,Gianluigi Donnarumma,NaN
5,6,Manchester Utd,9,5,1,3,15,14,1,16,1.78,16.2,13.6,2.6,0.29,W L W W W,73997,Bryan Mbeumo - 4,Altay Bayındır,NaN
6,7,Liverpool,9,5,0,4,16,14,2,15,1.67,15.8,12.0,3.8,0.42,W L L L L,60362,"Mohamed Salah, Cody Gakpo... - 3",Alisson,NaN
7,8,Aston Villa,9,4,3,2,9,8,1,15,1.67,7.1,11.8,-4.7,-0.52,D W W W W,41242,"Emi Buendía, Donyell Malen... - 2",Emiliano Martínez,NaN
8,9,Chelsea,9,4,2,3,17,11,6,14,1.56,13.4,13.1,0.3,0.03,L L W W L,39609,"Enzo Fernández, Moisés Caicedo - 3",Robert Sánchez,NaN
9,10,Crystal Palace,9,3,4,2,12,9,3,13,1.44,17.6,11.7,5.9,0.65,W W L D L,25068,Jean-Philippe Mateta - 5,Dean Henderson,NaN


In [ ]:
#!pip install selenium
# Web scrape data by web driver

from selenium import webdriver
import pandas as pd
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    # options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()
data=[]
#driver.get('https://fbref.com/en/comps/9/Premier-League-Stats#all_results2025-202691')
driver.get('https://fbref.com/en/comps/9/Premier-League-Stats')
data = driver.page_source

df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})
df

/tmp/ipython-input-2679361105.py:23: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(data, attrs={"id":"results2025-202691_overall"})


[    Rk            Squad  MP  W  D  L  GF  GA  GD  Pts  Pts/MP    xG   xGA  \
 0    1          Arsenal   9  7  1  1  16   3  13   22    2.44  14.6   5.2   
 1    2      Bournemouth   9  5  3  1  16  11   5   18    2.00  11.0  11.1   
 2    3        Tottenham   9  5  2  2  17   7  10   17    1.89   9.9  10.7   
 3    4       Sunderland   9  5  2  2  11   7   4   17    1.89   8.6  10.4   
 4    5  Manchester City   9  5  1  3  17   7  10   16    1.78  15.6   9.0   
 5    6   Manchester Utd   9  5  1  3  15  14   1   16    1.78  16.2  13.6   
 6    7        Liverpool   9  5  0  4  16  14   2   15    1.67  15.8  12.0   
 7    8      Aston Villa   9  4  3  2   9   8   1   15    1.67   7.1  11.8   
 8    9          Chelsea   9  4  2  3  17  11   6   14    1.56  13.4  13.1   
 9   10   Crystal Palace   9  3  4  2  12   9   3   13    1.44  17.6  11.7   
 10  11        Brentford   9  4  1  4  14  14   0   13    1.44  13.4  12.2   
 11  12    Newcastle Utd   9  3  3  3   9   8   1   12    1.33  